### Experimenting with SimpleRNN from https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN

In [1]:
import numpy as np
import tensorflow as tf

inputs = np.random.random([32, 10, 8]).astype(np.float32)

np.set_printoptions(precision=2)
inputs[0]

array([[0.19, 0.68, 0.08, 0.27, 0.95, 0.01, 0.27, 0.3 ],
       [0.85, 0.32, 0.3 , 0.81, 0.23, 0.73, 0.16, 0.18],
       [0.45, 0.73, 0.76, 0.28, 0.82, 0.57, 0.91, 0.25],
       [0.93, 0.65, 0.81, 0.82, 0.38, 0.32, 0.86, 0.61],
       [0.7 , 0.79, 0.65, 0.23, 0.27, 0.16, 0.07, 0.5 ],
       [0.12, 0.14, 0.65, 0.76, 0.52, 0.69, 0.97, 0.73],
       [0.46, 0.19, 0.88, 0.2 , 0.2 , 0.41, 0.61, 0.49],
       [0.14, 0.42, 0.93, 0.19, 0.19, 0.38, 0.82, 0.58],
       [0.88, 0.07, 0.71, 0.21, 0.96, 0.72, 0.48, 0.76],
       [0.34, 0.89, 0.63, 0.21, 0.59, 0.16, 0.03, 0.55]], dtype=float32)

In [2]:
inputs.shape

(32, 10, 8)

In [3]:
simple_rnn = tf.keras.layers.SimpleRNN(4)

output = simple_rnn(inputs)  
print('output.shape =', output.shape)
# The output has shape `[32, 4]`.

simple_rnn = tf.keras.layers.SimpleRNN(4, return_sequences=True, return_state=True)

whole_sequence_output, final_state = simple_rnn(inputs)
print('whole_sequence_output.shape =', whole_sequence_output.shape)
print('final_state.shape =', final_state.shape)
# whole_sequence_output has shape `[32, 10, 4]`.
# final_state has shape `[32, 4]`.

output.shape = (32, 4)
whole_sequence_output.shape = (32, 10, 4)
final_state.shape = (32, 4)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM, Dropout
from IPython.display import display

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=10, input_shape=(None, 2), activation='relu'))
model.summary()

In [ ]:
n_samples = 10000
n_periods = 20
phase_value = n_samples // n_periods

noise_scale = 0.2
t = np.arange(0, n_samples)
x = np.sin(2*np.pi*t/phase_value) + noise_scale * (np.random.rand(n_samples)-0.5) # + np.sin(0.5*t/phase_value)
df = pd.DataFrame(x)
display(df.head())

plt.subplots(figsize=(10, 6))
plt.plot(df[:2000])
plt.show()

In [ ]:
split_test_partition = int(0.8*n_samples)
train, test = df.values[0:split_test_partition, :], df.values[split_test_partition:, :]

#### Create a Sequence Data

In [ ]:
window_size = 10

def sequenceData(data, window_size):
    X, y = [], []
    for i in range(len(data)-window_size):
        d = i + window_size
        X.append(data[i:d, ])
        y.append(data[d, ])
    return np.array(X), np.array(y)

X_train, y_train = sequenceData(train, window_size)
X_test, y_test = sequenceData(test, window_size)

In [ ]:
X_train = X_train.reshape(-1, window_size, 1)
X_test = X_test.reshape(-1, window_size, 1)

In [ ]:
model = Sequential()
model.add(SimpleRNN(units=16, input_shape=(None, 1), activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1))

lr_scheduler = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001, decay_rate=1e-6, decay_steps=1, staircase=False)
opt = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)

model.compile(loss='mean_squared_error', optimizer=opt)
model.summary()

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
y_pred = model.predict(X_test)

fig, ax = plt.subplots(figsize=(20, 6))
plt.plot(y_test, 'o', markersize=2, label='actual')
plt.plot(y_pred, 'x', markersize=2, label='predict')
plt.plot(y_test - y_pred, '.', markersize=2, label='difference', color='darkslateblue')
ax.legend(loc='best')
plt.show()